The model

An Object Detection is a combination of two tasks:

regression of the bound-box coordinates
classification of the object label
This means that our model has two outputs: namely the object label and the object bound box. Therefore, the model must combine the tasks of classification and regression.

The Classifier

Let’s forget the bound box for a while and begin with a simpler classifier: